Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
import pandas as pd
import numpy as np

In [0]:
def wrangle(x):
    '''
    wrangle, train and test sets
    '''
    
    #prevent copy error
    #x = x.copy()
    
    #convert near zeros latitudes to zero
    x['latitude'] = x['latitude'].replace(-2e-08, 0)
    
    #replace zeros with NaN
    cols_w_zeros = ['longitude', 'latitude', 'gps_height', 
                   'construction_year', 'population']
    
    for c in cols_w_zeros:
        x[c] = x[c].replace(0, np.NaN)
        x[c+'_missing'] = x[c].isnull()
        
    #drop duplicate columns
    duplicates = ['payment_type', 'quantity_group']
    x = x.drop(columns=duplicates)
    
    #drop noise columns
    noise = ['recorded_by', 'id']
    x = x.drop(columns=noise)
    
    #convert date_recorded to datetime
    x['date_recorded'] = pd.to_datetime(x['date_recorded'], infer_datetime_format=True)
    
    #extract datetime compnents and drop datetime column
    x['yr_recorded'] = x['date_recorded'].dt.year
    x['month_recorded'] = x['date_recorded'].dt.month
    x['day_recorded'] = x['date_recorded'].dt.day
    x = x.drop(columns='date_recorded')
    
    #return the wrangel df
    return x

train = wrangle(train)
test = wrangle(test)

In [0]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=.8, test_size=.2,
                             stratify=train['status_group'], random_state=22)

In [0]:
#set target vector for models
target = 'status_group'

#get a df w/. all train features except target
train_features = train.drop(columns=[target])

#get a list of numeric columns
numeric_list = train_features.select_dtypes(include='number').columns.tolist()

#get a series with the cardinality of numeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

#get a list of categorical features w/ <= 50 categories
cat_list = cardinality[cardinality <= 50].index.tolist()

#combine the lists
features = cat_list + numeric_list

In [0]:
import category_encoders as ce
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [0]:
from sklearn.pipeline import make_pipeline

In [0]:
x_train = train[features]
y_train = train[target]

encoder = ce.OneHotEncoder(use_cat_names=True)
x_train_encoded = encoder.fit_transform(x_train)

In [0]:
x_train_encoded.sample(10)

,basin_Wami / Ruvu,basin_Pangani,basin_Rufiji,basin_Lake Rukwa,basin_Internal,basin_Ruvuma / Southern Coast,basin_Lake Victoria,basin_Lake Tanganyika,basin_Lake Nyasa,region_Morogoro,...,longitude,latitude,num_private,region_code,district_code,population,construction_year,yr_recorded,month_recorded,day_recorded
29983,0,0,0,0,0,1,0,0,0,0,...,35.010442,-10.944984,0,10,3,69.0,2000.0,2013,3,27
32463,1,0,0,0,0,0,0,0,0,0,...,38.864275,-6.830856,120,6,2,20.0,2010.0,2011,2,24
981,0,0,1,0,0,0,0,0,0,1,...,37.035866,-7.772331,0,5,3,200.0,1992.0,2011,3,30
10753,0,0,0,0,0,0,0,1,0,0,...,33.157027,-4.575374,0,14,1,NaN,NaN,2013,2,8
4048,0,1,0,0,0,0,0,0,0,0,...,37.315586,-3.403992,0,3,6,10.0,2012.0,2013,2,15
54360,0,0,0,1,0,0,0,0,0,0,...,32.072243,-8.692901,0,15,2,50.0,1986.0,2011,8,17
41077,0,1,0,0,0,0,0,0,0,0,...,37.477340,-3.280316,0,3,4,1.0,1972.0,2013,3,19
17908,0,1,0,0,0,0,0,0,0,0,...,36.842134,-3.491157,0,2,2,150.0,1980.0,2013,3,16
41469,0,1,0,0,0,0,0,0,0,0,...,37.314625,-3.319778,0,3,6,150.0,2012.0,2013,2,14
35682,0,0,0,0,0,0,0,1,0,0,...,30.584343,-3.684402,0,16,1,500.0,1993.0,2013,2,14


In [0]:
y_train_encoded = y_train.replace({'functional': 1, 'non functional': 2, 'functional needs repair': 3})

In [0]:
y_train_encoded.value_counts()

1    20646
2    14607
3     2763
Name: status_group, dtype: int64

In [0]:
pipeline = make_pipeline(
                        StandardScaler(),
                       SimpleImputer(), 
                       RandomForestClassifier())

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__max_depth': [10, 20, 30, 40],
    'randomforestclassifier__min_samples_leaf': [1, 3, 5]
}

search = RandomizedSearchCV(pipeline,
                            param_distributions=param_distributions,
                           n_iter=30,
                           cv=5,
                           scoring='neg_mean_absolute_error',
                           verbose=10,
                           return_train_score=True,
                           n_jobs=-1)

search.fit(x_train_encoded, y_train_encoded)

C:\Users\ricci\Anaconda3\envs\Py3.7\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 24 is smaller than n_iter=30. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 118 out of 120 | elapsed:  5.0min remaining:    5.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  5.1min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('standardscaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,
                                                            missing_values=nan,
                                                            strategy='mean',
                                                            verbose=0)),
                                             ('randomforestclassifier',
                      

In [0]:
print("Best Hyperparameters:", search.best_params_)
print("Cross-Validation MAE:", -search.best_score_)

Best Hyperparameters: {'simpleimputer__strategy': 'mean', 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__max_depth': 20}
Cross-Validation MAE: 0.25215697408405535
